In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorboardX import SummaryWriter
import time
import numpy as np

In [ ]:
# Configuration Variables
MEAN_PREPRO = [0.485, 0.456, 0.406]
STD_PREPRO = [0.229, 0.224, 0.225]
RESIZE_PREPRO = 256,256

TRAIN_BATCH_SIZE = 128
TRAIN_SHUFFLE = True
TRAIN_NUM_WORKERS = 8
TRAIN_PIN_MEMORY = True

VAL_BATCH_SIZE = 512
VAL_SHUFFLE = False
VAL_NUM_WORKERS = 4
VAL_PIN_MEMORY = True

INITIAL_LR = 1e-4
DEVICE_ID = 1

DATA_DIR = '/data/porn/binary/'
TRAINSET_ROOT = f'{DATA_DIR}/train'
TESTSET_ROOT = f'{DATA_DIR}/test'

TENSORBOARD_DIR = '/data/porn/tensorboard/resnet50-binary/'

In [ ]:
normalize = transforms.Normalize(
        mean=MEAN_PREPRO, std=STD_PREPRO
    )
prepro = transforms.Compose(
    [
        transforms.RandomResizedCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]
)
prepro_val = transforms.Compose(
    [transforms.Resize(RESIZE_PREPRO), transforms.ToTensor(), normalize]
)

In [4]:
train_dataset = ImageFolder(TRAINSET_ROOT, transform=prepro)
val_dataset = ImageFolder(TESTSET_ROOT, transform=prepro_val)

In [5]:
print(train_dataset)
print(train_dataset.classes)

Dataset ImageFolder
    Number of datapoints: 183210
    Root location: /data/porn/binary//train
    StandardTransform
Transform: Compose(
               RandomResizedCrop(size=(256, 256), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['nsfw', 'safe']


In [6]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, shuffle=VAL_SHUFFLE, num_workers=VAL_NUM_WORKERS, pin_memory=VAL_PIN_MEMORY, drop_last=False)

In [7]:
model = models.resnet50(pretrained=True)
model.fc = torch.nn.Linear(2048, len(train_dataset.classes))
DEVICE = f"cuda:{DEVICE_ID}"
model = model.to(DEVICE)
for p in model.parameters():
    p.requires_grad=False
for p in model.fc.parameters():
    p.requires_grad=True

In [8]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR)
criterion = nn.CrossEntropyLoss()

In [9]:
def plot_cm(classes, cm):
    fig, ax = plt.subplots(figsize=(len(classes),len(classes)))  
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax,fmt="d")
    # labels, title and ticks
    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels') 
    ax.set_title('Confusion Matrix') 
    ax.xaxis.set_ticklabels(classes,rotation=90)
    ax.yaxis.set_ticklabels(classes,rotation=0)
    return fig

In [10]:
def train(
    train_loader,
    model,
    criterion,
    optimizer,
    epoch,
    on_iteration=None,
):
    model = model.train()
    end = time.time()
    print("Start Training")
    avg_loss = 0
    for i, (inputs, labels) in enumerate(train_loader):
        print(f"{i/len(train_loader) * 100 : 2.2f}%", end="\r")
        iteration_time = time.time()
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
        if on_iteration is not None:
            on_iteration(iteration=i+epoch*len(train_loader), loss=loss, y_pred=outputs, y_true=labels)     
    return avg_loss/len(train_loader)

In [11]:
def validate(val_loader, 
             model, 
             criterion,
             print_freq=1000):
    model = model.eval()
    y_pred, y_true = [], []
    avg_loss = 0
    for i, (inputs, labels) in enumerate(val_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        with torch.no_grad():
            outputs = model(inputs)
            avg_loss += criterion(outputs, labels).item()
        _, indices = torch.max(outputs.cpu(), 1)
        y_pred.append(indices)
        y_true.append(labels.cpu().clone())
    return avg_loss/len(val_loader), torch.cat(y_pred), torch.cat(y_true)

In [12]:
def compute_porn_detection(epoch, y_pred, y_true):
    ones = torch.ones(len(y_pred))
    p_pred = (y_pred == 1) | (y_pred == 3)
    p_truth = (y_true == 1) | (y_true == 3)
    logger.add_pr_curve("Eval/Prec_recall", p_truth, p_pred, epoch)

In [13]:
def validation_logs(epoch, loss_avg, y_pred, y_true):
    logger.add_scalar("Loss/Avg_Val", loss_avg, epoch)
    logger.add_scalar("Eval/Precision", (y_pred==y_true).sum()/float(len(y_pred)), epoch)
    compute_porn_detection(epoch, y_pred, y_true)
    cm = confusion_matrix(y_pred, y_true)
    cm_image = plot_cm(train_dataset.classes, cm)
    logger.add_figure("Eval/Confusion Matrix", cm_image, epoch)

In [14]:
def on_iteration_logs(iteration, loss, y_pred, y_true):
    l = loss.item()
    if iteration%50 == 0:
        logger.add_scalar("Loss/Train", l, iteration)
        print(
                f"{iteration}/{len(train_loader)} \t"
                f"Loss {l}"
            )

In [15]:
logger = SummaryWriter(TENSORBOARD_DIR)

In [16]:
for i in range(10):
    val_loss, y_pred, y_true = validate(val_loader, model, criterion)
    validation_logs(i, val_loss, y_pred, y_true)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)

Start Training
0/1431 	Loss 0.5941365957260132
50/1431 	Loss 0.4584013819694519
100/1431 	Loss 0.406158447265625
150/1431 	Loss 0.3455604016780853
200/1431 	Loss 0.3352430760860443
250/1431 	Loss 0.30484941601753235
300/1431 	Loss 0.37025219202041626
350/1431 	Loss 0.24612148106098175
400/1431 	Loss 0.25426894426345825
450/1431 	Loss 0.193686842918396
500/1431 	Loss 0.2738170623779297
550/1431 	Loss 0.27412036061286926
600/1431 	Loss 0.23882710933685303
650/1431 	Loss 0.2792549431324005
700/1431 	Loss 0.26615047454833984
750/1431 	Loss 0.28119412064552307
800/1431 	Loss 0.26657208800315857
850/1431 	Loss 0.1947667896747589
900/1431 	Loss 0.2480231076478958
950/1431 	Loss 0.23323087394237518
1000/1431 	Loss 0.26831427216529846
1050/1431 	Loss 0.22298309206962585
1100/1431 	Loss 0.2509477138519287
1150/1431 	Loss 0.2654440402984619
1200/1431 	Loss 0.27652451395988464
1250/1431 	Loss 0.24988488852977753
1300/1431 	Loss 0.22201591730117798
1350/1431 	Loss 0.23847956955432892
1400/1431 	Los

In [17]:
for i, child in enumerate(model.children()):
    if i > 6:
        for p in child.parameters():
            p.requires_grad=True

In [19]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE//2, shuffle=TRAIN_SHUFFLE, num_workers=TRAIN_NUM_WORKERS, pin_memory=True, drop_last=True)

In [20]:
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.1)

In [21]:
for i in range(10,20):
    val_loss, y_pred, y_true = validate(val_loader, model, criterion)
    validation_logs(i, val_loss, y_pred, y_true)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)

Start Training
28650/2862 	Loss 0.18004454672336578
28700/2862 	Loss 0.20700326561927795
28750/2862 	Loss 0.1323094666004181
28800/2862 	Loss 0.15843237936496735
28850/2862 	Loss 0.319430947303772
28900/2862 	Loss 0.25684744119644165
28950/2862 	Loss 0.1464371681213379
29000/2862 	Loss 0.11895129084587097
29050/2862 	Loss 0.19154097139835358
29100/2862 	Loss 0.14050543308258057
29150/2862 	Loss 0.24416416883468628
29200/2862 	Loss 0.1800231635570526
29250/2862 	Loss 0.23812174797058105
29300/2862 	Loss 0.14606578648090363
29350/2862 	Loss 0.2400764524936676
29400/2862 	Loss 0.1481853574514389
29450/2862 	Loss 0.05916711688041687
29500/2862 	Loss 0.2874000370502472
29550/2862 	Loss 0.20592767000198364
29600/2862 	Loss 0.1193256601691246
29650/2862 	Loss 0.21155336499214172
29700/2862 	Loss 0.14782389998435974
29750/2862 	Loss 0.21358148753643036
29800/2862 	Loss 0.2594754695892334
29850/2862 	Loss 0.09838778525590897
29900/2862 	Loss 0.1941651701927185
29950/2862 	Loss 0.174310714006423

In [24]:
m = models.resnext101_32x8d()

In [25]:
m

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1

In [ ]:
for i, child in enumerate(model.children()):
    if i > 3:
        for p in child.parameters():
            p.requires_grad=True
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=INITIAL_LR*0.01)

In [ ]:
for i in range(20,30):
    val_loss, y_pred, y_true = validate(val_loader, model, criterion)
    validation_logs(i, val_loss, y_pred, y_true)
    loss = train(
                    train_loader,
                    model,
                    criterion,
                    optimizer,
                    i,
                    on_iteration=on_iteration_logs,
                )
    logger.add_scalar("Loss/Avg_Train", loss, i)